Link to the original question can be found [here](http://adventofcode.com/2017/day/6)

First let's retrieve our data.

We'll define an input file called _inputFile.txt_ which is located in our current directry. Then we'll use the _System.IO_ library to access the file's contents using _File.ReadAllLines_. Since this is an IO operation we should wrap it in a function to work better with the rest of our functional code.

In [1]:
open System.IO
open System


type Item = int*int
type Message = Item seq

let inputFile = "problem6.txt"

let parseFile (input: string) : Message = 
    File.ReadAllText(input)
        |> fun fileString -> fileString.Split()
        |> Array.toSeq
        |> Seq.map System.Int32.Parse 
        |> Seq.mapi (fun i v -> i, v)
        
parseFile inputFile

seq [(0, 11); (1, 11); (2, 13); (3, 7); ...]

So here our _parseFile_ function accepts a string and converts it to a _Message_, which we've defined as a sequence of _Items,_ or int pairs. 

Making types like Item and Message helps us because they add type safety to our functions as well as make our type declarations neater and more readable. _Message_ looks a whole  lot better than _Seq<int*int>_ later on when reviewing. Although one can argue that the abstraction makes it harder to determine what's entering and leaving our functions. 

Now let's define a function that can accept a message and find the highest number in the message.

In [2]:
let findMax (fullMap: Message) : Item =
    let folder ((key1, val1): Item) ((key2, val2): Item) : Item =
        if val1 >= val2 then 
            key1, val1 
        else 
            key2, val2
    
    Seq.fold folder (0, 0) fullMap
    
[(0, 13); (1, 6); (2, 20);] |> findMax

(2, 20)

Note that _folder_ in _findMax_ compares val1 to val2 so the left side of the list is given priority over the right in the case of the two values being the same size, as described in the original function. If values on the right are chosen over values on the left that _may_ influence our final answer. 

In [3]:
let filterer ((key1, _): Item) ((key2, _): Item) : bool = 
        if key1 = key2 then true else false
        
filterer (2, 5) (2, 10)
//filterer (2, 5) (3, 5)

true

_filterer_ is just a helper function that checks if two Items have the same index

In [4]:
let shift (offset: int) (items: Message) : Message = 
    let first = 
        items
            |> Seq.filter (fun (key, _) -> key > offset)
    
    let second = 
        items
            |> Seq.filter (fun (key, _) -> key <= offset)
            
    Seq.append first second
    
shift (2) (Seq.ofList [(0, 0); (1, 1); (2, 2); (3, 3); ])

seq [(3, 3); (0, 0); (1, 1); (2, 2)]

_shift_ rotates a sequences to make the element _after_ the index passed in the first element

In [5]:
let rec pluser (countdown: int) (items: Message) : Message = 
    if countdown = 0 then
        items
    else
        let new_head = items |> Seq.head |> fun (key, value) -> (key, value + 1)
        let new_message = Seq.append (Seq.tail items) (Seq.singleton new_head)
        pluser (countdown - 1) new_message
        
pluser 50 [(0, 0); (1, 1); (2, 2);]

seq [(2, 18); (0, 17); (1, 18)]

_pluser_ takes an integer and spreads it across each element till the integer runs out.

In [6]:
let setToZero (key: int) (message: Message) : Message =
    let zeroed =
        message
            |> Seq.find (fun (x, y) -> x = key)
            |> fun (x, y) -> (x, 0)
            
    let filtered =
        message
            |> Seq.filter (fun (x, y) -> not (x = key))
            
    Seq.append (Seq.singleton zeroed) filtered
        |> Seq.sortBy (fun (x, y) -> x)
        
setToZero 0 [(0, 11); (1, 6); (2, 0);]

seq [(0, 0); (1, 6); (2, 0)]

_setToZero_ accepts an index and and message and sets the value at that index to zero

In [7]:
let advance (input: Message) : Message =

    let key, value = findMax input
    let zeroed = setToZero key input
    let shifted = shift key zeroed  
    
    shifted
    |> pluser value
    |> Seq.sortBy (fun (x, y) -> x)
    
[(0, 11); (1, 6); (2, 0);] |> advance |> advance |> advance |> advance |> advance

seq [(0, 6); (1, 3); (2, 8)]

Okay, so here we've defined an advance function that actually updates our sequence. Let's go over what's happening.

First, we find the key and value of the largest item. We conveniently call them _key_ and _value._

We set the largest item back to zero.

We shift the list so the formerly highest item is last.

Then we distribute the highest value using pluser.

Finally we resort the list.

In [8]:
let messageCheck (mess1: Message) (mess2: Message) : bool = 
        (Set.ofSeq mess1) = (Set.ofSeq mess2)
            
messageCheck ([(0, 11); (1, 6); (2, 0);] |> Seq.ofList) ([(0, 11); (1, 6); (2, 0);] |> Seq.ofList)

true

Our last helper function is _messageCheck_ which checks two messages for equality by converting thing to sets and doing a comparison. This works because our messages contain immutable tuples. 

In [ ]:
let rec iterate (cache: Message seq) : int =
    
    let nextMessage = cache |> Seq.head |> advance
    
    match Seq.exists (messageCheck nextMessage) cache with
    | true ->
        Seq.length cache
    | false -> 
        cache
            |> Seq.append (Seq.singleton nextMessage)
            |> iterate

inputFile |> parseFile |> Seq.singleton |> iterate

Finally we have our _iterate_ function. This is where we implement our pesky recursion. Our iterate function accepts a sequnce of messages we call cache. It takes the head, advances it, and then checks to see if the new message is already in the cache. If it is, it returns the length of the cache, else it adds the new message to the front and keeps going.

Seq.exists is a built in sequence function that basically checks if any value in a sequence satisfies some given function.

In [10]:
let adventOfCode6Part1 (sourceString: string) : int =
    sourceString
        |> parseFile 
        |> Seq.singleton
        |> iterate
        
adventOfCode6Part1 inputFile

The value or constructor 'iterate' is not defined.

_adventOfCode6Part1_ is a neat little function wrapper than encapsulates everything we've done so far. Accepting a string, it opens and parses the fill, converts to a sequence of int pair sequences, and iterates till a result is found. 

I was hoping to avoid the use of explicit recursion again, but I dont think this answer is too bad.